In [658]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from collections import Counter
import seaborn as sns
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.corpus import sentiwordnet
from nltk.util import ngrams
stop_words = set(stopwords.words('english')) 
lemmatizer = nltk.WordNetLemmatizer()

In [659]:
def flat_list(var):
    flat = [item for sublist in var for item in sublist]
    return flat

In [ ]:
os.makedirs('./Results/MK', exist_ok = True)

In [660]:
df = pd.read_csv('./Data/Koneski_final_finished_dataset.csv')

In [661]:
df.tail()

,Unnamed: 0,Godina_od,Godina_do,Zbirka,Zbirka_broj,Podzbirka,Podzbirka_broj,Pesna_Broj,Pesna_Ime,Pesna,Stih_Broj,Stih,Pesna_eng,Stih_eng,Unnamed: 14,Unnamed: 15
9379,9379,1993,1993,ЦРН ОВЕН (1993),7,ОTPОР,1,36,СПРОТИ СВЕТИ АТАНАС 1993,"СПРОТИ СВЕТИ АТАНАС 1993\nЗимски дену,\nми нос...",6,Како да се поништи ужасот,"OPPOSITE SAINT ATANAS 1993\nWinter day,\nyou b...",How to undo the horror,NaN,NaN
9380,9380,1993,1993,ЦРН ОВЕН (1993),7,ОTPОР,1,36,СПРОТИ СВЕТИ АТАНАС 1993,"СПРОТИ СВЕТИ АТАНАС 1993\nЗимски дену,\nми нос...",7,со тоа што човек,"OPPOSITE SAINT ATANAS 1993\nWinter day,\nyou b...",by being a man,NaN,NaN
9381,9381,1993,1993,ЦРН ОВЕН (1993),7,ОTPОР,1,36,СПРОТИ СВЕТИ АТАНАС 1993,"СПРОТИ СВЕТИ АТАНАС 1993\nЗимски дену,\nми нос...",8,не може да го сфати?,"OPPOSITE SAINT ATANAS 1993\nWinter day,\nyou b...",can not understand it?,NaN,NaN
9382,9382,1993,1993,ЦРН ОВЕН (1993),7,ОTPОР,1,36,СПРОТИ СВЕТИ АТАНАС 1993,"СПРОТИ СВЕТИ АТАНАС 1993\nЗимски дену,\nми нос...",9,Проклета слабост да се зборне.,"OPPOSITE SAINT ATANAS 1993\nWinter day,\nyou b...",Damn weakness to speak.,NaN,NaN
9383,9383,1993,1993,ЦРН ОВЕН (1993),7,ОTPОР,1,36,СПРОТИ СВЕТИ АТАНАС 1993,"СПРОТИ СВЕТИ АТАНАС 1993\nЗимски дену,\nми нос...",10,"Поживинченост, ако молчиш.","OPPOSITE SAINT ATANAS 1993\nWinter day,\nyou b...","Liveliness, if you are silent.",NaN,NaN


In [662]:
df_stih_in_pesna_count = df.groupby(['Pesna_Broj', 'Zbirka_broj'], as_index=False).last()

## Number of stihs in pesna

In [663]:
df_stih_in_pesna_count.head()

,Pesna_Broj,Zbirka_broj,Unnamed: 0,Godina_od,Godina_do,Zbirka,Podzbirka,Podzbirka_broj,Pesna_Ime,Pesna,Stih_Broj,Stih,Pesna_eng,Stih_eng,Unnamed: 14,Unnamed: 15
0,1,1,5126,1984,1984,ЧЕШМИТЕ (1984),ДОЈРАНСКИ ВЕТРИШТА,2,АНА,АНА\nНа сpоменоt на Ана Пенинgtон\nТаа беше за...,16,Но секој носел своја судбина.,ANA\nIn memory of Anne Pennington\nShe was for...,But everyone had their own destiny.,None,None
1,1,2,5430,1987,1987,ПОСЛАНИЕ (1987),ПОСЛАНИЕ,1,ПОЕЗИЈО,"ПОЕЗИЈО\nПоезијо,\nми отвораш пат во староста,...",12,само со едното и другото.,"POETRY\nPoetry,\nyou open the way for me in ol...",only with both.,None,None
2,1,3,6269,1988,1988,ЦРКВА (1988),ЦРКВА (1988),1,ЦРКВА,ЦРКВА\nНека е проклет часот кога ми кажаа\nдек...,123,твоите свети камења.,CHURCH\nCursed be the hour when they told me\n...,your sacred stones.,The old men jump to the old men,None
3,1,4,7320,1989,1989,ЗЛАТОВРВ (1989),СКРЖАВИ PЕСНИ,4,ГЛЕДАЊЕ ВО ФИЛЏАН,ГЛЕДАЊЕ ВО ФИЛЏАН\nЕве една стара жена\nзабрад...,14,на твојот живот.,LOOKING IN A CUP\nHere is an old woman\nlined ...,sitting by the window,roared in the dark,None
4,1,5,8151,1989,1989,СЕИЗМОГРАФ (1989),ЗАНАЕT,3,МАКЕДОНСКИ ПОЕТИ,МАКЕДОНСКИ ПОЕТИ\nКаква судбина!\nДа ти се чин...,11,исполнето.,"MACEDONIAN POETS\nWhat a destiny!\nYes, it see...",fulfilled.,None,None


In [664]:
df_stih_in_pesna_count.to_csv('./Results/number_of_stih_in_pesna.csv')

In [665]:
pesni = df['Pesna_eng']
stihovi = df['Stih_eng']
godina_od = df['Godina_od']
godina_do = df['Godina_do']

In [666]:
pesni_list = set(pesni.to_list())
pesni_list = (list(pesni_list))
pesni_list = [x for x in pesni_list if str(x) != 'nan']
stihovi_list = stihovi.to_list()
godina_od_list = godina_od.to_list()
godina_do_list = godina_do.to_list()

In [667]:
len(godina_do_list)

9384

# Stihovi

## Word count with lematization and removed stop words

In [668]:
stihovi_cleaned = []
stihovi_sentences = []
ids = []
for i, stih in enumerate(stihovi_list):
    if type(stih) is not float and type(stih) is not int:
        stih = stih.lower()
        stih = nltk.word_tokenize(stih)
        stih = [re.sub(r'[^\w\s]','',word) for word in stih]
        stih = [word for word in stih if not word in stop_words]
        stih = [word for word in stih if word.isalpha()]
        stih = [lemmatizer.lemmatize(each_word) for each_word in stih]
        if len(stih) == 0:
            continue
        stihovi_sentences.append(' '.join(stih))
        stihovi_cleaned.append(stih)
        ids.append(i)

In [669]:
godina_od_list = df.iloc[ids, 1]
godina_do_list = df.iloc[ids, 2]
ime_na_pesni = df.iloc[ids, :]['Pesna_Ime']
ime_na_zbirka = df.iloc[ids, :]['Zbirka']
ime_na_podzbirka = df.iloc[ids, :]['Podzbirka']

In [670]:
sentences_df = pd.DataFrame({'sentence': stihovi_sentences, 
                             'year_from': godina_od_list, 
                             'year_to': godina_do_list, 
                             'song_name':ime_na_pesni, 
                             'zbirka_name':ime_na_zbirka, 
                             'podzbirka_name':ime_na_podzbirka})

In [671]:
sentences_df.to_csv('./Results/sentences.csv')

### Average words per stih

In [672]:
number_of_words = 0
for stih in stihovi_cleaned:
    number_of_words += len(stih)
print(number_of_words/len(stihovi_cleaned))
stihovi_cleaned_flat = flat_list(stihovi_cleaned)

2.89791916330755


In [673]:
stihovi_cleaned_flat = flat_list(stihovi_cleaned)

In [674]:
stih_count = Counter(stihovi_cleaned_flat)

In [675]:
stih_count_df = pd.DataFrame(stih_count.most_common())
stih_count_df.columns = ['words', 'count']
stih_count_df.to_csv('./Results/stih_word_count.csv')

In [676]:
stih_count_df_10 = stih_count_df[stih_count_df['count'] > 10]
stih_count_df_10.to_csv('./Results/stih_word_count_10.csv')

## Word count without lematization and removed stop words

In [677]:
stihovi_cleaned = []
stihovi_sentences = []
ids = []
for i, stih in enumerate(stihovi_list):
    if type(stih) is not float and type(stih) is not int:
        stih = stih.lower()
        stih = nltk.word_tokenize(stih)
        stih = [re.sub(r'[^\w\s]','',word) for word in stih]
        stih = [word for word in stih if not word in stop_words]
        stih = [word for word in stih if word.isalpha()]
        if len(stih) == 0:
            continue
        stihovi_sentences.append(' '.join(stih))
        stihovi_cleaned.append(stih)
        ids.append(i)

In [678]:
godina_od_list = df.iloc[ids, 1]
godina_do_list = df.iloc[ids, 2]
ime_na_pesni = df.iloc[ids, :]['Pesna_Ime']
ime_na_zbirka = df.iloc[ids, :]['Zbirka']
ime_na_podzbirka = df.iloc[ids, :]['Podzbirka']

In [679]:
sentences_df = pd.DataFrame({'sentence': stihovi_sentences, 
                             'year_from': godina_od_list, 
                             'year_to': godina_do_list, 
                             'song_name':ime_na_pesni, 
                             'zbirka_name':ime_na_zbirka, 
                             'podzbirka_name':ime_na_podzbirka})

In [680]:
sentences_df.to_csv('./Results/sentences_without_lem_without_stop.csv')

### Average words per stih

In [681]:
number_of_words = 0
for stih in stihovi_cleaned:
    number_of_words += len(stih)
print(number_of_words/len(stihovi_cleaned))
stihovi_cleaned_flat = flat_list(stihovi_cleaned)

2.89791916330755


In [682]:
stihovi_cleaned_flat = flat_list(stihovi_cleaned)

In [683]:
stih_count = Counter(stihovi_cleaned_flat)

In [684]:
stih_count_df = pd.DataFrame(stih_count.most_common())
stih_count_df.columns = ['words', 'count']

In [685]:
stih_count_df.to_csv('./Results/stih_word_count_without_lem.csv')

In [686]:
stih_count_df_10 = stih_count_df[stih_count_df['count'] > 10]
stih_count_df_10.to_csv('./Results/stih_word_count_without_lem_10.csv')

## Word count without lematization and without removed stop words

In [687]:
stihovi_cleaned = []
stihovi_sentences = []
ids = []
for i, stih in enumerate(stihovi_list):
    if type(stih) is not float and type(stih) is not int:
        stih = stih.lower()
        stih = nltk.word_tokenize(stih)
        stih = [re.sub(r'[^\w\s]','',word) for word in stih]
        stih = [word for word in stih if word.isalpha()]
        if len(stih) == 0:
            continue
        stihovi_sentences.append(' '.join(stih))
        stihovi_cleaned.append(stih)
        ids.append(i)

In [688]:
godina_od_list = df.iloc[ids, 1]
godina_do_list = df.iloc[ids, 2]
ime_na_pesni = df.iloc[ids, :]['Pesna_Ime']
ime_na_zbirka = df.iloc[ids, :]['Zbirka']
ime_na_podzbirka = df.iloc[ids, :]['Podzbirka']

In [689]:
sentences_df = pd.DataFrame({'sentence': stihovi_sentences, 
                             'year_from': godina_od_list, 
                             'year_to': godina_do_list, 
                             'song_name':ime_na_pesni, 
                             'zbirka_name':ime_na_zbirka, 
                             'podzbirka_name':ime_na_podzbirka})

In [690]:
sentences_df.to_csv('./Results/sentences_without_lem_with_stopwords.csv')

### Average words per stih 

In [691]:
number_of_words = 0
for stih in stihovi_cleaned:
    number_of_words += len(stih)
avg_words_per_stih = round(number_of_words/len(stihovi_cleaned))
print(round(number_of_words/len(stihovi_cleaned)))

6


In [692]:
stihovi_cleaned_flat = flat_list(stihovi_cleaned)

In [693]:
stih_count = Counter(stihovi_cleaned_flat)

In [694]:
stih_count_df = pd.DataFrame(stih_count.most_common())
stih_count_df.columns = ['words', 'count']
stih_count_df.to_csv('./Results/stih_word_count_without_lem_with_stopwords.csv')

In [695]:
stih_count_df_10 = stih_count_df[stih_count_df['count'] > 10]
stih_count_df_10.to_csv('./Results/stih_word_count_lem_with_stopwords_10.csv')

# Pesni

## Word count with lematization and removed stop words

In [696]:
pesni_cleaned = []
pesni_sentences = []
ids = []
for i, pesna in enumerate(pesni_list):
    if type(pesna) is not float:
        pesna = pesna.lower()
        pesna = nltk.word_tokenize(pesna)
        pesna = [re.sub(r'[^\w\s]','',word) for word in pesna]
        pesna = [word for word in pesna if not word in stop_words]
        pesna = [word for word in pesna if word.isalpha()]
        pesna = [lemmatizer.lemmatize(each_word) for each_word in pesna]
        if len(pesna) == 0:
            continue
        pesni_sentences.append(' '.join(pesna))
        pesni_cleaned.append(pesna)
        ids.append(i)

In [697]:
godina_od_list = df.iloc[ids, 1]
godina_do_list = df.iloc[ids, 2]
ime_na_pesni = df.iloc[ids, :]['Pesna_Ime']
ime_na_zbirka = df.iloc[ids, :]['Zbirka']
ime_na_podzbirka = df.iloc[ids, :]['Podzbirka']

In [698]:
sentences_df = pd.DataFrame({'pesna': pesni_sentences, 
                             'year_from': godina_od_list, 
                             'year_to': godina_do_list, 
                             'song_name':ime_na_pesni, 
                             'zbirka_name':ime_na_zbirka, 
                             'podzbirka_name':ime_na_podzbirka})

In [699]:
sentences_df.to_csv('./Results/pesni_lem_without_stopwords.csv')

### Average words per pesna

In [700]:
number_of_words = 0
pesna_len = []
for pesna in pesni_cleaned:
    number_of_words += len(pesna)
    pesna_len.append(len(pesna))
print(number_of_words/len(pesni_cleaned))

43.4367816091954


In [701]:
pesna_len_counter = Counter(pesna_len)

In [702]:
pesna_len_counter_df = pd.DataFrame.from_dict(pesna_len_counter, orient='index').reset_index()

In [703]:
pesna_len_counter_df = pesna_len_counter_df.rename(columns={'index':'words_in_pesna', 0:'count'})

In [704]:
pesna_len_counter_df.to_csv('./Results/word_count_in_pesna_lem_without_stop.csv')

## Word count without lematization and removed stop words

In [705]:
pesni_cleaned = []
pesni_sentences = []
ids = []
for i, pesna in enumerate(pesni_list):
    if type(pesna) is not float:
        pesna = pesna.lower()
        pesna = nltk.word_tokenize(pesna)
        pesna = [re.sub(r'[^\w\s]','',word) for word in pesna]
        pesna = [word for word in pesna if not word in stop_words]
        pesna = [word for word in pesna if word.isalpha()]
        if len(pesna) == 0:
            continue
        pesni_sentences.append(' '.join(pesna))
        pesni_cleaned.append(pesna)
        ids.append(i)

In [706]:
godina_od_list = df.iloc[ids, 1]
godina_do_list = df.iloc[ids, 2]
ime_na_pesni = df.iloc[ids, :]['Pesna_Ime']
ime_na_zbirka = df.iloc[ids, :]['Zbirka']
ime_na_podzbirka = df.iloc[ids, :]['Podzbirka']

In [707]:
sentences_df = pd.DataFrame({'pesna': pesni_sentences, 
                             'year_from': godina_od_list, 
                             'year_to': godina_do_list, 
                             'song_name':ime_na_pesni, 
                             'zbirka_name':ime_na_zbirka, 
                             'podzbirka_name':ime_na_podzbirka})

In [708]:
sentences_df.to_csv('./Results/pesni_without_lem_without_stopwords.csv')

### Average words per pesna

In [709]:
number_of_words = 0
pesna_len = []
for pesna in pesni_cleaned:
    number_of_words += len(pesna)
    pesna_len.append(len(pesna))
print(number_of_words/len(pesni_cleaned))

43.4367816091954


In [710]:
pesna_len_counter = Counter(pesna_len)

In [711]:
pesna_len_counter_df = pd.DataFrame.from_dict(pesna_len_counter, orient='index').reset_index()

In [712]:
pesna_len_counter_df = pesna_len_counter_df.rename(columns={'index':'words_in_pesna', 0:'count'})

In [713]:
pesna_len_counter_df.to_csv('./Results/word_count_in_pesna_no_lem_without_stop.csv')

## Word count without lematization and without removed stop words

In [714]:
pesni_cleaned = []
pesni_sentences = []
ids = []
for i, pesna in enumerate(pesni_list):
    if type(pesna) is not float:
        pesna = pesna.lower()
        pesna = nltk.word_tokenize(pesna)
        pesna = [re.sub(r'[^\w\s]','',word) for word in pesna]
        pesna = [word for word in pesna if word.isalpha()]
        if len(pesna) == 0:
            continue
        pesni_sentences.append(' '.join(pesna))
        pesni_cleaned.append(pesna)
        ids.append(i)

In [715]:
godina_od_list = df.iloc[ids, 1]
godina_do_list = df.iloc[ids, 2]
ime_na_pesni = df.iloc[ids, :]['Pesna_Ime']
ime_na_zbirka = df.iloc[ids, :]['Zbirka']
ime_na_podzbirka = df.iloc[ids, :]['Podzbirka']

In [716]:
sentences_df = pd.DataFrame({'pesna': pesni_sentences, 
                             'year_from': godina_od_list, 
                             'year_to': godina_do_list, 
                             'song_name':ime_na_pesni, 
                             'zbirka_name':ime_na_zbirka, 
                             'podzbirka_name':ime_na_podzbirka})

In [717]:
sentences_df.to_csv('./Results/pesni_without_lem_with_stopwords.csv')

### Average words per pesna

In [718]:
number_of_words = 0
pesna_len = []
for pesna in pesni_cleaned:
    number_of_words += len(pesna)
    pesna_len.append(len(pesna))
avg_words_in_pesna = round(number_of_words/len(pesni_cleaned))
print(round(number_of_words/len(pesni_cleaned)))

90


In [719]:
pesna_len_counter = Counter(pesna_len)

In [720]:
pesna_len_counter_df = pd.DataFrame.from_dict(pesna_len_counter, orient='index').reset_index()

In [721]:
pesna_len_counter_df = pesna_len_counter_df.rename(columns={'index':'words_in_pesna', 0:'count'})

In [722]:
pesna_len_counter_df.to_csv('./Results/word_count_in_pesna_no_lem_with_stop.csv')

### Average stih per pesna

In [723]:
stih_in_pesna = df_stih_in_pesna_count['Stih_Broj']

In [724]:
import statistics
import math
avg_stih_in_pesna = round(statistics.mean(stih_in_pesna))
print(avg_stih_in_pesna)

15


# Report

In [725]:
with open("Report.txt", 'w') as report:
    report.write("==========================REPORT=============================")
    report.write(f"AVERAGE NUMBER OF STIHS IN PESNA {avg_stih_in_pesna}")
    report.write(f"AVERAGE NUMBER OF ZBOROVI IN PESNA {avg_words_in_pesna}")
    report.write(f"AVERAGE NUMBER OF ZBOROVI VO STIH {avg_words_per_stih}")
    report.write("==========================REPORT=============================")